## The simpliest usage example of py_boost

### Installation (if needed)

**Note**: replace cupy-cuda110 with your cuda version!!!

In [1]:
# !pip install cupy-cuda110 py-boost

### Imports

In [2]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import joblib
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting 

### Generation of dummy regression data

In [3]:
%%time
X, y = make_regression(150000, 100, n_targets=10, random_state=42)
X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 1.88 s, sys: 1.3 s, total: 3.19 s
Wall time: 829 ms


### Training a GBDT model

The only argument required here is a loss function. It, together with the input target shape, determines the task type. The loss function can be passed as a Loss instance or using a string alias:

* ***'mse'*** for the regression/multitask regression
* ***'msle'*** for the regression/multitask regression
* ***'bce'*** for the binary/multilabel classification
* ***'crossentropy'*** for the multiclassification

Training is simply done by calling the .fit metod. Possible argumentsare the following:

* ***'X'*** 
* ***'y'*** 
* ***'sample_weight'*** 
* ***'eval_sets'***  
A validation set is passed as a list of dicts with possible keys ['X', 'y', 'sample_weight']. Note: if multiple valid sets are passed, the best model is selected using the last one.

#### The example below illustrates how to train a simple regression task.

In [4]:
%%time
model = GradientBoosting('mse')

model.fit(X, y[:, 0], eval_sets=[{'X': X_test, 'y': y_test[:, 0]},])

[09:41:29] Stdout logging level is INFO.
[09:41:29] GDBT train starts. Max iter 100, early stopping rounds 100
[09:41:29] Iter 0; Sample 0, rmse = 173.6750220764608; 
[09:41:29] Iter 10; Sample 0, rmse = 133.1954898620719; 
[09:41:29] Iter 20; Sample 0, rmse = 107.86658300143122; 
[09:41:29] Iter 30; Sample 0, rmse = 90.08216999114677; 
[09:41:29] Iter 40; Sample 0, rmse = 76.44548323474514; 
[09:41:29] Iter 50; Sample 0, rmse = 65.61048065141752; 
[09:41:30] Iter 60; Sample 0, rmse = 56.80157577360109; 
[09:41:30] Iter 70; Sample 0, rmse = 49.57707316105171; 
[09:41:30] Iter 80; Sample 0, rmse = 43.60320091852412; 
[09:41:30] Iter 90; Sample 0, rmse = 38.69696218785459; 
[09:41:30] Iter 99; Sample 0, rmse = 34.99120585299892; 
CPU times: user 4.04 s, sys: 814 ms, total: 4.85 s
Wall time: 3.63 s


### Traininig a GBDT model in a multiregression case

Each of built-in loss functions has its own default metric, so metric definition is optional. 
If you need to specify the evaluation metric, you can pass a Metric instance or use a string alias.

#### Default metrics:

* ***'rmse'*** is the default for the ***'mse'*** loss
* ***'rmsle'*** is the default for the  ***'msle'*** loss
* ***'bce'*** is the default for the ***'bce'*** loss
* ***'crossentropy'*** is the default for the ***'crossentropy'*** loss

#### Non-default metrics:

* ***'r2'*** for the regression/multitask regression
* ***'auc'*** for the binary classification
* ***'accuracy'*** for any classification task
* ***'precision'*** for any classification task
* ***'recall'*** for any classification task
* ***'f1'*** for any classification task

It is possible to specify other common GBDT hyperparameters as shown below.

#### The following example demonstrates how to train a model for a multioutput regression task (no extra definition needed to switch the task to multioutput one, you just need to pass a multidimensional target).

In [5]:
%%time
model = GradientBoosting('mse', 'r2_score',
                         ntrees=1000, lr=.01, verbose=100, es=200, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10, min_gain_to_split=0, 
                         max_bin=256, max_depth=6)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[09:41:30] Stdout logging level is INFO.
[09:41:30] GDBT train starts. Max iter 1000, early stopping rounds 200
[09:41:30] Iter 0; Sample 0, R2_score = 0.00838493389477495; 
[09:41:33] Iter 100; Sample 0, R2_score = 0.5168165390880501; 
[09:41:35] Iter 200; Sample 0, R2_score = 0.7243208313385177; 
[09:41:37] Iter 300; Sample 0, R2_score = 0.8328614385331654; 
[09:41:39] Iter 400; Sample 0, R2_score = 0.8949967440653801; 
[09:41:41] Iter 500; Sample 0, R2_score = 0.9320127030761451; 
[09:41:43] Iter 600; Sample 0, R2_score = 0.9546913138388898; 
[09:41:45] Iter 700; Sample 0, R2_score = 0.9687615526168845; 
[09:41:47] Iter 800; Sample 0, R2_score = 0.9776369907528174; 
[09:41:50] Iter 900; Sample 0, R2_score = 0.9833275841564569; 
[09:41:52] Iter 999; Sample 0, R2_score = 0.9870190620496355; 
CPU times: user 20.1 s, sys: 2.95 s, total: 23.1 s
Wall time: 21.9 s


## Inference

#### Prediction can be done via calling the .predict method

In [6]:
%%time
preds = model.predict(X_test)

preds.shape

CPU times: user 1 s, sys: 481 ms, total: 1.48 s
Wall time: 1.5 s


(50000, 10)

In [7]:
preds

array([[-240.17752 , -145.03291 , -275.28513 , ..., -137.55057 ,
        -210.75629 , -233.15964 ],
       [-113.514046, -114.34615 ,  -58.754765, ..., -127.49755 ,
        -116.32571 ,  -21.29363 ],
       [ -35.181435,  -55.067272,  142.86133 , ...,   20.998077,
         -21.99269 , -208.16821 ],
       ...,
       [ -80.83802 ,  133.6094  ,   85.85005 , ...,  225.99036 ,
          36.656986,   13.258009],
       [  -5.558326,  141.5677  ,  251.22258 , ...,  153.28246 ,
         179.13031 ,  210.6762  ],
       [ -25.74225 ,   29.600954,  164.6824  , ...,   89.410904,
          18.127083,   -0.798545]], dtype=float32)

#### Prediction for certan iterations can be done via calling the .predict_staged method

In [8]:
%%time
preds = model.predict_staged(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 255 ms, sys: 219 ms, total: 474 ms
Wall time: 484 ms


(3, 50000, 10)

#### Tree leaves indicies prediction for certan iterations can be done via calling the .predict_leaves method

In [9]:
%%time
preds = model.predict_leaves(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 6.66 ms, sys: 10.8 ms, total: 17.5 ms
Wall time: 16.8 ms


(3, 50000, 1)

In [10]:
preds.T[0]

array([[11, 46, 10],
       [54, 46, 28],
       [32, 46, 55],
       ...,
       [54, 53, 10],
       [27, 46, 20],
       [60, 46, 27]], dtype=uint32)

#### Feature importances

In [11]:
model.get_feature_importance()

array([  40.,   52.,   31.,   49.,   57.,   62., 5790.,   44.,   48.,
         70.,   49.,   51.,   52.,   41.,   34., 5901., 5427.,   38.,
         33., 5442.,   33.,   39.,   55.,   62.,   39.,   37.,   45.,
         59.,   43.,   45.,   57.,   62.,   34.,   45.,   29.,   60.,
       6040.,   46.,   43.,   37.,   39.,   40.,   46.,   42.,   52.,
         61.,   40.,   44.,   46.,   55.,   44.,   45., 5918.,   38.,
         42.,   73.,   49.,   34.,   37.,   48.,   55.,   37.,   38.,
         51.,   46.,   39.,   42.,   43.,   42.,   58.,   40.,   41.,
         46.,   39.,   49.,   49.,   58.,   39.,   37.,   51.,   61.,
         37.,   50.,   51.,   66.,   30., 5607., 3529.,   41., 5747.,
         53., 6175.,   46.,   56.,   45.,   40.,   38.,   51.,   41.,
         65.], dtype=float32)

#### The trained model can be saved as pickle for inference

In [12]:
joblib.dump(model, '../data/temp_model.pkl')

new_model = joblib.load('../data/temp_model.pkl')
new_model.predict(X_test)

array([[-240.17752 , -145.03291 , -275.28513 , ..., -137.55057 ,
        -210.75629 , -233.15964 ],
       [-113.514046, -114.34615 ,  -58.754765, ..., -127.49755 ,
        -116.32571 ,  -21.29363 ],
       [ -35.181435,  -55.067272,  142.86133 , ...,   20.998077,
         -21.99269 , -208.16821 ],
       ...,
       [ -80.83802 ,  133.6094  ,   85.85005 , ...,  225.99036 ,
          36.656986,   13.258009],
       [  -5.558326,  141.5677  ,  251.22258 , ...,  153.28246 ,
         179.13031 ,  210.6762  ],
       [ -25.74225 ,   29.600954,  164.6824  , ...,   89.410904,
          18.127083,   -0.798545]], dtype=float32)